In Julia, `1` and `1.0` are different values, because they have different types:

In [1]:
@show typeof(1);
@show typeof(1.0);

typeof(1) = Int64


typeof(1.0) = Float64


The standard choice for floating-point values is `Float64`, which is double precision using 64 binary bits. We can see all the bits by using `bitstring`.

In [2]:
bitstring(1.0)

"0011111111110000000000000000000000000000000000000000000000000000"

The first bit determines the sign of the number:

:::
```{tip}
Square brackets concatenate the contained values into vectors.
```

In [3]:
[bitstring(1.0), bitstring(-1.0)]

2-element Vector{String}:
 "0011111111110000000000000000000000000000000000000000000000000000"
 "1011111111110000000000000000000000000000000000000000000000000000"

The next 11 bits determine the exponent (scaling) of the number, and so on.

In [4]:
[bitstring(1.0), bitstring(2.0)]

2-element Vector{String}:
 "0011111111110000000000000000000000000000000000000000000000000000"
 "0100000000000000000000000000000000000000000000000000000000000000"

The sign bit, exponent, and significand in {eq}`floatpoint` are all directly accessible.

In [5]:
x = 3.14
@show sign(x), exponent(x), significand(x);

(sign(x), exponent(x), significand(x)) = (1.0, 1, 1.57)


In [6]:
x = x / 8
@show sign(x), exponent(x), significand(x);

(sign(x), exponent(x), significand(x)) = (1.0, -2, 1.57)


The spacing between floating-point values in $[2^n,2^{n+1})$ is $2^n \epsilon_\text{mach}$, where $\epsilon_\text{mach}$ is machine epsilon. You can get its value from the `eps` function in Julia. By default, it returns the value for double precision.

```{tip}
To call a function, including `eps`, you must use parentheses notation, even when there are no input arguments.
```

In [7]:
eps()

2.220446049250313e-16

Because double precision allocates 52 bits to the significand, the default value of machine epsilon is $2^{-52}$.

In [8]:
log2(eps())

-52.0

The spacing between adjacent floating-point values is proportional to the magnitude of the value itself. This is how relative precision is kept roughly constant throughout the range of values. You can get the adjusted spacing by calling `eps` with a value.

In [9]:
eps(1.618)

2.220446049250313e-16

In [10]:
eps(161.8)

2.842170943040401e-14

In [11]:
nextfloat(161.8)

161.80000000000004

In [12]:
ans - 161.8

2.842170943040401e-14

A common mistake is to think that $\epsilon_\text{mach}$ is the smallest floating-point number. It's only the smallest *relative to 1*. The correct perspective is that the scaling of values is limited by the exponent, not the mantissa. The actual range of positive values in double precision is

In [13]:
@show floatmin(), floatmax();

(floatmin(), floatmax()) = (2.2250738585072014e-308, 1.7976931348623157e308)


For the most part you can mix integers and floating-point values and get what you expect.

In [14]:
1/7

0.14285714285714285

In [15]:
37.3 + 1

38.3

In [16]:
2^(-4)

0.0625

There are some exceptions. A floating-point value can't be used as an index into an array, for example, even if it is numerically equal to an integer. In such cases you use `Int` to convert it.

In [17]:
@show 5.0, Int(5.0);

(5.0, Int(5.0)) = (5.0, 5)


If you try to convert a noninteger floating-point value into an integer you get an `InexactValue` error. This occurs whenever you try to force a type conversion that doesn't make clear sense.